<a href="https://colab.research.google.com/github/SruthiM-10/Irrigation-Scheduling-Recommendations-And-Dataset/blob/main/dataset_processing/(2)_initial_analysis_and_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from openai import OpenAI
import re
import json
import requests
import os

In [ ]:
! pip install -q langchain_community langchain_openai faiss-cpu langchain langchain_core langgraph pinecone PyPDF2 doi2pdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/20

In [ ]:
import time
import uuid
import requests
from langchain_community.document_loaders import WebBaseLoader
from typing import Dict, List, Any, Tuple, Optional, Union
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
import pickle
from langchain_core.pydantic_v1 import Field, conlist, BaseModel

import warnings

warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    module='langchain_openai.chat_models.base'
)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# Analyzing Per Feature (Test 1)

In [ ]:
all_samples = pd.read_csv('/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/all_feature_samples.csv')

In [ ]:
samples = all_samples.loc[all_samples["content source"] == "pdf"]
# doi and file_path are different?? but data corresponds correctly to the file path
# still need to fix duplicate papers problem for this one

In [ ]:
info["feature"]

"""
Logically relevant list --
plant_type, location, irrigation_scheduling_method, number_of_plants (farm_size), evapotranspiration

Location quite important, supposedly inferring features like local weather, local topography, etc. from the tests done at this location
Perhaps also just try to get specific lat/lon, altitude

Also maybe log what methods were used to calculate/measure evapotranspiration
"""

,feature
11,plant_type
10,location
1,irrigation_scheduling_method
17,plant_yield
7,amount_irrigated
19,water_productivity
12,number_of_plants
9,irrigation_time
5,evapotranspiration_level
8,number_of_irrigation_events


In [ ]:
samples["plant_type"].describe()

,plant_type
count,411
unique,92
top,Potato (Solanum tuberosum L.)
freq,69


In [ ]:
samples["plant_type"]

# Check its the same but just the species name is specified and if it isn't
# If there's more then one plant type (comma separated or with a "+" ?), explode that row and duplicate the value in the other features if it doesn't it isn't specified in that feature for which plant type it is for

,plant_type
0,Cucumber (Zhongnong 26)
1,Cucumber (Zhongnong 26)
2,Cucumber (Zhongnong 26)
3,Wheat (Triticum aestivum L.)
4,Wheat (Triticum aestivum L.)
...,...
542,"Corn, Soybean"
543,"Corn, Soybean"
544,"Corn, Soybean"
545,"Corn, Soybean"


In [ ]:
samples["location"].describe()

,location
count,396
unique,79
top,"Agricultural Experiment Station, Damascus, Syria"
freq,51


In [ ]:
samples["location"]
# perhaps remove the specific station name?, keep only city/state/province/country
# Or don't need to standardize, keep specific names for better learning?

,location
0,Xiaotangshan National Precision Agriculture Ex...
1,Xiaotangshan National Precision Agriculture Ex...
2,Xiaotangshan National Precision Agriculture Ex...
3,"Awash Basin, Ethiopia"
4,"Melkassa Agricultural Research Center, Awash B..."
...,...
542,Southwest Michigan
543,Southwest Michigan
544,Southwest Michigan
545,Southwest Michigan


In [ ]:
samples["irrigation_scheduling_method"].describe()

,irrigation_scheduling_method
count,377
unique,190
top,Furrow irrigation
freq,44


In [ ]:
samples["irrigation_scheduling_method"]

# too generic, so too many that seem to be the same, when they are actually different, need to be more specific
# note that a lot of data is being lost in tables (confirmed)

"""
List of expected methods (generally):
furrow irrigation, micro-irrigation, sprinkler, rainfed irrigation
methods of drip irrigation - sustained, regulated (stage-based (stage of plant growth), on film, partial root-zone, subsurface)
Soil depletion (different types)

combination of sensors + sensor thresholds -- soil, evapotranspiration, weather, etc
"""

,irrigation_scheduling_method
0,Soil moisture sensor-based method (T-VWC)
1,Accumulated radiation combined with soil moist...
2,Crop evapotranspiration estimated method using...
3,Deficit irrigation
4,Deficit irrigation
...,...
542,Maximum Allowable Depletion (MAD) 60%
543,Maximum Allowable Depletion (MAD) 50%
544,Maximum Allowable Depletion (MAD) 30%
545,Producer’s Management


In [ ]:
samples["evapotranspiration_level"].describe()

,evapotranspiration_level
count,155
unique,63
top,>2000 mm year-1
freq,39


In [ ]:
samples["evapotranspiration_level"]

,evapotranspiration_level
0,ETd = 4.63-4.66 mm
1,ETd = 4.66 mm
2,ETd = 2.94 mm
3,"100% ETc, 85% ETc, 70% ETc, 55% ETc, 40% ETc"
4,"100% ETc, 85% ETc, 70% ETc, 55% ETc, 40% ETc"
...,...
542,NaN
543,NaN
544,NaN
545,NaN


# Analyzing Per Feature (Test 2)

In [ ]:
all_samples = pd.read_csv('/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/all_feature_samples (2).csv')

In [ ]:
samples = all_samples.loc[all_samples["content source"] == "pdf"]

In [ ]:
samples = samples.loc[(all_samples["irrigation_scheduling_method"] != "NA") & (all_samples["irrigation_scheduling_method"].notna())]
samples = samples.loc[(all_samples["plant_yield"] != "NA") & (all_samples["plant_yield"].notna())]
samples

,Unnamed: 0,irrigation_scheduling_method,sensor_type,sensor_threshold,number_of_sensors,evapotranspiration_level,daily_drainage,amount_irrigated,number_of_irrigation_events,irrigation_time,...,plant_height,water_productivity,doi_url,file_path,doi,references,processed_at,content source,relevance_score,relevance_dict
0,0,Soil moisture sensor-based method (T-VWC),Soil moisture sensor,VWC max = 0.507 m3 m−3,1,"ETd = 4.63 mm (beginning), 3.79 mm (middle), 3...",0.34-0.43 L plant−1,"1.70-2.00 L plant−1 (beginning), 2.00 L plant−...",12,07:00–18:00,...,NaN,17.37 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
1,1,Accumulated radiation combined with soil moist...,Soil moisture sensor,Accumulated radiation = 0.8 MJ m−2,1,"ETd = 4.66 mm (beginning), 3.03 mm (middle), 1...","0.40 L plant−1 (beginning), 0.34 L plant−1 (mi...","2.03 L plant−1 (beginning), 1.40 L plant−1 (mi...","14 (beginning), 9 (middle), 5 (end)",NaN,...,NaN,20.52 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
2,2,Crop evapotranspiration estimated method using...,NaN,ETc h ≥ 0.4 mm,NaN,"ETd = 2.94 mm (beginning), 2.17 mm (middle), 2...","0.26 L plant−1 (beginning), 0.36 L plant−1 (mi...","1.29 L plant−1 (beginning), 1.12 L plant−1 (mi...","9 (beginning), 9 (middle), 10 (end)",NaN,...,NaN,25.07 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
3,3,Soil water balance method,"Neutron probe, gravimetric method",50% depletion of total available soil moisture,NaN,"100% ETc, 85% ETc, 70% ETc, 55% ETc, 40% ETc",NaN,"417.2 mm (100% ETc), 341 mm (85% ETc), 286 mm ...","14 (2021/2022), 11 (2022/2023)",December to March (2021/2022 and 2022/2023),...,NaN,1.42 kg m−3 (70% ETc),https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:01:02,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
10,10,Regulated Deficit Irrigation,NaN,NaN,NaN,55% ETc,NaN,NaN,NaN,Every two days at 8 o'clock,...,0.490 m,NaN,https://doi.org/10.1093/jxb/erv034,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,[{'title': 'Effect of water stress at differen...,2025-09-12 04:06:41,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,579,Deficit Irrigation (Soil Texture: Loam),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,13.51%,https://doi.org/10.3390/plants13050640,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/plants13050640,[{'title': 'Evaluating the Effect of Deficit I...,2025-09-13 00:00:24,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
580,580,Deficit Irrigation (Soil Texture: Sandy),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8.83%,https://doi.org/10.3390/plants13050640,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/plants13050640,[{'title': 'Evaluating the Effect of Deficit I...,2025-09-13 00:00:24,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
581,581,Deficit Irrigation (Soil Available Nitrogen Co...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.11%,https://doi.org/10.3390/plants13050640,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/plants13050640,[{'title': 'Evaluating the Effect of Defic

In [ ]:
samples = samples[["irrigation_scheduling_method", "plant_yield", "location", "latitude", "longitude", "altitude", "plant_type", "water_productivity", "file_path", "doi", "doi_url", "processed_at", "content source", "relevance_score"]]
samples

,irrigation_scheduling_method,plant_yield,location,latitude,longitude,altitude,plant_type,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),17.37 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),20.52 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),25.07 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...","Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),1.42 kg m−3 (70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
10,Regulated Deficit Irrigation,2.406 kg m-2,"Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,Deficit Irrigation (Soil Texture: Loam),-8.73%,"Xinjiang, China",NaN,NaN,NaN,NaN,13.51%,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/plants13050640,https://doi.org/10.3390/plants13050640,2025-09-13 00:00:24,pdf,9
580,Deficit Irrigation (Soil Texture: Sandy),-13.98%,"Xinjiang, China",NaN,NaN,NaN,NaN,8.83%,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/plants13050640,https://doi.org/10.3390/plants13050640,2025-09-13 00:00:24,pdf,9
581,Deficit Irrigation (Soil Available Nitrogen Co...,-14%,"Xinjiang, China",NaN,NaN,NaN,NaN,3.11%,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/plants13050640,https://doi.org/10.3390/plants13050640,2025-09-13 00:00:24,pdf,9
599,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)","Korla, Xinjiang, China",40°53′N,86°56′E,900 m,Gossypium hirstum L.,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9


In [ ]:
relative_pattern = r"(^-?\d+(\.\d+)?%$|highest|lowest|increase|decrease|decline|reduction|greater|lower|minimal|no significant|effect)"
def is_relative_description(yield_value):
    return bool(re.search(relative_pattern, yield_value.lower()))

# Filter out rows with relative descriptions
samples_clean = samples[~samples['plant_yield'].apply(is_relative_description)].reset_index(drop=True)
samples_clean

,irrigation_scheduling_method,plant_yield,location,latitude,longitude,altitude,plant_type,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),17.37 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),20.52 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),25.07 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...","Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),1.42 kg m−3 (70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,2.406 kg m-2,"Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,Non-irrigated,"12,108 kg/ha for corn, 3,569 kg/ha for soybean",Dry corner of each site,NaN,NaN,NaN,"Corn, Soybean",NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9
242,Non-stressed (I1),"4,974 kg/ha (2006-2007), 4,802 kg/ha (2007-2008)","Shiraz Agricultural Research Station, Iran",29°50'N,52°46'E,1810 m,Winter wheat,0.66 to 1.22 kg m/C03,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.2166/ws.2021.246,https://doi.org/10.2166/ws.2021.246,2025-09-12 23:48:15,pdf,9
243,Post-anthesis water stress (I2),"4,272 kg/ha (2006-2007), 4,485 kg/ha (2007-2008)","Shiraz Agricultural Research Station, Iran",29°50'N,52°46'E,1810 m,Winter wheat,0.63 to 1.08 kg m/C03,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.2166/ws.2021.246,https://doi.org/10.2166/ws.2021.246,2025-09-12 23:48:15,pdf,9
244,Timing of last irrigation,1669-2405 kg/ha,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,NaN,NaN,Cotton (Gossypium hirsutum L.),0.45-0.71 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8


In [ ]:
plant_yield = samples_clean["plant_yield"].unique()
print(f"After {len(plant_yield)}")
plant_yield

After 196


array(['66.77 t ha−1', '58.98 t ha−1', '64.54 t ha−1',
       '5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% ETc), 4,555 kg ha−1 (70% ETc), 3,475 kg ha−1 (55% ETc), 2,510 kg ha−1 (40% ETc)',
       '2.406 kg m-2', '2.540 kg m-2', '2.701 kg m-2', '3.06 kg m-2',
       '6.1 t ha-1', '6.4 t ha-1', '0.2 t ha-1', '3.7 t ha-1',
       '3.5 t ha-1', '2.2 t ha-1', '9.9 t ha-1', '5.1 t ha-1',
       '1.4 t ha-1', '164.9 t ha-1', '752.3 t ha-1', '5.5 t ha-1',
       '6.0 t ha-1', '3.6 t ha-1', '58.6 t ha-1', '26.1 t ha-1',
       '2.3 t ha-1', '2.6 t ha-1', '34.7 t ha-1', '33.8 t ha-1',
       '18.15 qt/ha', '24.84 qt/ha', '24.51 qt/ha', '21.99 qt/ha',
       '17.36 qt/ha', '3.060 kg m-2', '3384.3 kg/ha', '2626.0 kg/ha',
       '3042.8 kg/ha', '2736.5 kg/ha', '3087.4 kg/ha', '2388.0 kg/ha',
       '2271.3 kg/ha', '2228.0 kg/ha', '2968.7 kg/ha', '4037.2 kg/ha',
       '9731.43 kg ha-1', '5455 kg/ha', '5228 kg/ha',
       '5.150 t ha−1 (N-S), 3.894 t ha−1 (S), 2.950 t ha−1 (HS)',
       '4.158 t ha

In [ ]:
samples_clean.to_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (1).csv")